Import neccessary packages


In [5]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils.image_utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [9]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '../dataset/RiceLeafs_Resized_1000/'
width = 256
height = 256
depth = 3


Function to convert images to array


In [7]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, default_image_size)
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None


Fetch images from directory


In [10]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir:
        # remove .DS_Store from list
        if directory == ".DS_Store":
            root_dir.remove(directory)

    for plant_folder in root_dir:
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")

        for disease_folder in plant_disease_folder_list:
            # remove .DS_Store from list
            if disease_folder == ".DS_Store":
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(
                f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

            for single_plant_disease_image in plant_disease_image_list:
                if single_plant_disease_image == ".DS_Store":
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")
except Exception as e:
    print(f"Error : {e}")


[INFO] Loading images ...
[INFO] Processing BrownSpot ...
[INFO] Processing Healthy ...
[INFO] Processing Hispa ...
[INFO] Processing LeafBlast ...
[INFO] Processing BrownSpot ...
[INFO] Processing Healthy ...
[INFO] Processing Hispa ...
[INFO] Processing LeafBlast ...
[INFO] Image loading completed


Get Size of Processed Image


In [11]:
image_size = len(image_list)


Transform Image Labels uisng [Scikit Learn](http://scikit-learn.org/)'s LabelBinarizer


In [12]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer, open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)


Print the classes


In [13]:
print(label_binarizer.classes_)


['BrownSpot' 'Healthy' 'Hispa' 'LeafBlast']


In [14]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0


In [15]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(
    np_image_list, image_labels, test_size=0.2, random_state=42)


[INFO] Spliting data to train, test


In [16]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2, horizontal_flip=True,
    fill_mode="nearest")


In [17]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))


Model Summary


In [18]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 activation (Activation)     (None, 256, 256, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 85, 85, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 64)        1

In [20]:
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the network
print("[INFO] training network...")


[INFO] training network...


In [22]:
history = model.fit(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
)


Epoch 1/25
24/24 [==============================] - 231s 10s/step - loss: 0.8312 - accuracy: 0.3893 - val_loss: 0.6498 - val_accuracy: 0.1818
Epoch 2/25
24/24 [==============================] - 173s 7s/step - loss: 0.6433 - accuracy: 0.4513 - val_loss: 2.4407 - val_accuracy: 0.2727
Epoch 3/25
24/24 [==============================] - 175s 7s/step - loss: 0.5628 - accuracy: 0.4908 - val_loss: 2.4021 - val_accuracy: 0.3030
Epoch 4/25
24/24 [==============================] - 153s 6s/step - loss: 0.5510 - accuracy: 0.4803 - val_loss: 1.4757 - val_accuracy: 0.2121
Epoch 5/25
24/24 [==============================] - 153s 6s/step - loss: 0.5471 - accuracy: 0.4684 - val_loss: 1.1604 - val_accuracy: 0.2374
Epoch 6/25
24/24 [==============================] - 190s 8s/step - loss: 0.5365 - accuracy: 0.4674 - val_loss: 2.9737 - val_accuracy: 0.2424
Epoch 7/25
24/24 [==============================] - 162s 7s/step - loss: 0.5364 - accuracy: 0.4684 - val_loss: 0.7983 - val_accuracy: 0.2727
Epoch 8/25
2

Plot the train and val curve


In [23]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
# Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
# Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()


KeyError: 'acc'

Model Accuracy


In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")


Save model using Pickle


In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model, open('cnn_model.pkl', 'wb'))
